<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    <font color='red'>To begin: Click anywhere in this cell and press <kbd>Run</kbd> on the menu bar. This executes the current cell and then highlights the next cell. There are two types of cells. A <i>text cell</i> and a <i>code cell</i>. When you <kbd>Run</kbd> a text cell (<i>we are in a text cell now</i>), you advance to the next cell without executing any code. When you <kbd>Run</kbd> a code cell (<i>identified by <span style="font-family: courier; color:black; background-color:white;">In[ ]:</span> to the left of the cell</i>) you advance to the next cell after executing all the Python code within that cell. Any visual results produced by the code (text/figures) are reported directly below that cell. Press <kbd>Run</kbd> again. Repeat this process until the end of the notebook. <b>NOTE:</b> All the cells in this notebook can be automatically executed sequentially by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart and Run All</kbd>. Should anything crash then restart the Jupyter Kernal by clicking <kbd>Kernel</kbd><font color='black'>→</font><kbd>Restart</kbd>, and start again from the top.
        
</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
<img src="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/cimcb_logo.png?raw=true" width="180px" align="right" style="padding: 20px">

<h1> ANNSigSig_MTBLS136 </h1>

<br>
<br>
<br>
<p style="text-align: justify"> The study used in this tutorial has been previously published by  <a href="https://europepmc.org/abstract/MED/30830410">Stevens et al. (2018)</a>, and the deconvolved and annotated data file deposited at the Metabolights data repository. The data can be accessed directly via its study ID: <a href="https://www.ebi.ac.uk/metabolights/MTBLS136">MTBLS136</a>. This workflow requires data to be formatted as a Microsoft Excel file, using the Tidy Data framework (i.e. each column is a variable, and row is an observation). As such, the Excel file contains a Data Sheet and Peak Sheet. The Data Sheet contains all the metabolite concentrations and metadata associated with each observation (requiring the inclusion of the columns: Idx, SampleID, and Class). The Peak Sheet contains all the metadata pertaining to each measured metabolite (requiring the inclusion of the columns: Idx, Name, and Label). Please inspect the Excel file <a href="https://github.com/CIMCB/MetabComparisonBinaryML/blob/master/dynamic/data/MTBLS136.xlsx?raw=true">MTBLS136.xlsx</a> used in this workflow before proceeding.</p>

<p style="text-align: justify">This is a serum LC-MS dataset consisting of 949 named metabolites. The primary outcome for this paper was estrogen-only (E; n=332) vs. estrogen plus progestin (E+P; n=337) vs. non-users of post-menopausal hormone therapy regimes (Control; n=667). For the purpose of this study, we compare only the E vs. E+P in a binary discriminant analysis.</p>
<br>


</ol> 
</div>

<div style="text-align: justify; padding:5px; background-color:rgb(252, 253, 255); border: 1px solid lightgrey; padding-left: 1em; padding-right: 1em;">
    
<h1>ANN-SS Workflow </h1>
<br>
<p style="text-align: justify">This Jupyter Notebook implements the complete workflow for creating, optimising, and evaluating a 2 layer artificial neural network with Layer 1 consisting of multiple neurons (n = 2 to 6) with a sigmoidal activation, and Layer 2 (output layer) consisting of a single neuron with a sigmoidal activation function (ANN-SS). <b style="text-align: justify"> ANN was implemented using <a href="https://keras.io">Keras</a> with a <a href="http://deeplearning.net/software/theano/">Theano backend.</a></b></p>
    
<i style="text-align: justify"> Please refer to the 'cimcb' package documentation for further details regarding this specific implementation: <a href="https://cimcb.github.io/cimcb">https://cimcb.github.io/cimcb</a></i><br>
 
<br>

<b style="text-align: justify"> ANN uses the following Hyperparameter(s):</b>
<ul style="list-style-type: square;">
    <li><code>learning_rate</code>: the parameter that controls the step-size in updating the weights (default=0.01) </li>
    <li><code>n_neurons</code>: the number of neurons in the hidden layer (default=2)</li>
    <li><code>epochs</code>: the number of iterations in the model training (default=100) </li>
    <li><code>momentum</code>: a value that alters the learning rate schedule, whereby increasing the learning rate when the error cost gradient continue in the same direction (default=0.5)</li>
    <li><code>decay</code>: a value that alters the learning rate schedule, whereby decreasing the learning rate after each epoch/iteration (default=0)</li>
     <li><code>loss</code>: the function used to calculate the error of the model during the model training process known as backpropagation (default='binary_crossentropy')</li>
</ul>
<i style="text-align: justify">The purpose of each hyperparameter is explained here: <a href="http://neuralnetworksanddeeplearning.com/">Nielsen (2015)</a></i>
 
<br>
<br>

<p style="text-align: justify">Preliminary analysis indicated, for the metabolomics data sets used in this study, that varying hyperparameters <code>momentum</code> and <code>decay</code> had little impact on performance, thus they were kept constant at their default values. Additionally, it was observed that fixing the number of <code>epochs</code> to 400 proved effective across most of the data sets. Thus hyperparameter optimisation was reduced to a grid search across <code>n_neurons = [2,3,4,5,6]</code> and <code>learning_rate = [0.0001,0.001,0.01,0.1,1] </code>. After the number of neurons is chosen, the learning rate was fine-tuned as appropriate using a linear search.</p>

<br>

<b style="text-align: justify"> The notebook workflow is broken into the following steps:</b>

<ol>
    <li><b><i>Import Packages</i></b>: First, the Python packages required for this workflow need to be imported (<a href="http://www.numpy.org/"><code>numpy</code></a>, <a href="https://pandas.pydata.org/"><code>pandas</code></a>, and <a href="https://cimcb.github.io/cimcb"><code>cimcb</code></a>).
</li>
    <li><b><i>Load Data & Peak Sheet:</i></b> From the Excel spreadsheet, import the Data and Peak spreadsheets and create two respective <a href="https://pandas.pydata.org/">Pandas</a> tables: <code>DataTable</code> and <code>PeakTable</code>.</li>
    <li><b><i>Extract X & Y:</i></b> Next, we reduce the data in <code>DataTable</code> to include only those observations needed for the binary comparison and create a new table: <code>DataTable2</code>. We define one column of the data table to be the "outcome" variable <code>Outcomes</code>, and convert the class labels in this column to a binary outcome vector <code>Y</code>, where <code>1</code> is the positive outcome, and <code>0</code> the negative outcome (eg. case=1 & control=0). A new variable <code>peaklist</code> is created to hold the names (M1...Mn) of the metabolites to be used in the discriminant analysis. To create an independent dataset to evaluate, <a href="https://scikit-learn.org/stable/">scikit-learn</a> module's <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html"><code>train_test_split()</code></a> function is used. The data is split into 2/3rd training (<code>DataTrain</code> and <code>YTrain</code>), and 1/3rd test (<code>DataTest</code> and <code>YTest</code>). The metabolite data corresponding to <code>peaklist</code> is extracted from <code>DataTrain</code> and placed in a matrix <code>XTrain</code>. The <code>XTrain</code> matrix is log-transformed and auto-scaled, with missing values imputed using k-nearest neighbours (k=3). Then the metabolite data corresponding to <code>peaklist</code> is extracted from <code>DataTest</code> and placed in a matrix <code>XTest</code>. The <code>XTest</code> matrix is log-transformed and auto-scaled (using mu and sigma from <code>XTrain</code>), with missing values imputed using k-nearest neighbours (k=3).
    <li><b><i>Hyperparameter Optimisation:</i></b> Here, we use the helper function <code>cb.cross_val.KFold()</code> to carry out 5-fold cross-validation of a set of ANN models (ANN-LS) configured with different values for learning rate (0.001 to 1) and number of neurons (2 to 6). This helper function is generally applicable, and the values being passed to it are: 
    <ul>
    <li>The class of model to be created by the function, <code>cb.model.NN_SigmoidSigmoid</code>.</li>
        <li>The metabolite matrix, <code>XTknn</code>, and binary outcome vector, <code>Y</code>.</li>
        <li>A dictionary, <code>param_dict</code>, describing key:value pairs where the key is a parameter that is passed to the model, and the value is a list of values to be passed to that parameter.</li>
        <li>The number of folds in the cross-validation, <code>folds</code>, and the number of monte carlo repetitions of the k-fold CV, <code>n_mc</code>.</li></ul>
When <code>cv.run()</code> followed by <code>cv.plot(metric='r2q2')</code> are run the predictive ability of the multiple models across the hyperparameter grid search (<code>n_neurons</code> vs. <code>learning_rate</code>) are displayed in the form of heatmaps representing the parametric performance values $R^2$, $Q^2$ and $|R^2 - Q^2|$. These heatmaps are interactively linked to a scatter plot of $|R^2 - Q^2|$ vs. $Q^2$ and line plots of $R^2$ & $Q^2$ vs <code>n_neurons</code> and <code>learning_rate</code>. If the function <code>cv.plot(metric='auc')</code> is run the predictive ability of the models is presented as measures of the area under the ROC curve, $AUC(full)$ & $AUC(cv)$, as a nonparametric alternative to $R^2$ & $Q^2$. These multiple plots are used to aid in selecting the optimal hyperparameter values.</li>
    <li><b><i>Build Model & Evaluate:</i></b> Here, we use the class <code>cb.model.NN_SigmoidSigmoid()</code> to building a ANN-SS model using the optimal hyperparameter values determined in step 4. The model is trained on the training dataset, <code>XTrainKnn</code>, and tested on the independent test dataset, <code>XTestKnn</code>. Next, the trained model's <code>.evaluate()</code> method is used to visualise model performance for both the training and independent test dataset using: a <a href="https://www.data-to-viz.com/graph/violin.html">violin plot</a> showing the distributions of negative and positive responses as violin and box-whisker plots; a <a href="https://books.google.com.au/books?id=7WBMrZ9umRYC">probability density function</a> plot for each response type, and a <a href="https://doi.org/10.1007/s11306-012-0482-9">ROC curve</a> that displays the curve for the training dataset (green) and test dataset (yellow).
   <li><b><i>Bootstrap Evaluation:</i></b> Finally, to create an estimate of the robustness and a measure of generalised predictive ability of this model we perform  <a href="https://link.springer.com/article/10.1007%2FBF00058655">bootstrap aggregation</a> (Bagging) using the helper function <code>cb.bootstrap.Per()</code> with 100 boostrapped models. This generates a population of 100 model predictions for both the training set (in-bag prediction - IB) and the holdout test set (out-of-bag - OOB) from the full dataset, with the metabolite matrix, <code>XBootKnn</code>, and binary outcome vector, <code>Y</code>. These predictions are visualised with a box-violin and probability density function plot for the aggregate model. The ROC curve displays the curve for the training dataset (green) and test dataset (yellow) from section 5 with 95% confidence intervals (light green band = IB & light yellow band = OOB).
  <li><b><i>Export Results:</i></b> Exporting the model evaluation results as an Excel spreadsheet.</li>
</ol> 

</div>

### 1. Import Packages

In [1]:
import numpy as np
import pandas as pd
import cimcb as cb
from sklearn.model_selection import train_test_split

print('All packages successfully loaded')

Using Theano backend.


All packages successfully loaded


### 2. Load Data & Peak Sheet

In [2]:
home = 'data/' 
file = 'MTBLS136.xlsx'  

DataTable,PeakTable = cb.utils.load_dataXL(home + file, DataSheet='Data', PeakSheet='Peak') 

Loadings PeakFile: Peak
Loadings DataFile: Data
Data Table & Peak Table is suitable.
TOTAL SAMPLES: 1649 TOTAL PEAKS: 949
Done!


### 3. Extract X & Y

In [3]:
# Clean PeakTable and Extract PeakList
PercMiss = PeakTable['Perc_missing']  
PeakTableClean = PeakTable[(PercMiss < 20)] 
PeakList = PeakTableClean['Name']  

# Select Subset of Data
DataTable2 = DataTable[(DataTable.Class == 1) | (DataTable.Class == 0)]

# Create a Binary Y Vector 
Outcomes = DataTable2['Class']
Y = Outcomes.values 

# Split Data into Train (2/3) and Test (1/3)
DataTrain, DataTest, YTrain, YTest = train_test_split(DataTable2, Y, test_size=1/3, stratify=Y, random_state=85)

# Extract Train Data 
XTrain = DataTrain[PeakList]                                    
XTrainLog = np.log(XTrain)                                          
XTrainScale, mu, sigma = cb.utils.scale(XTrainLog, method='auto', return_mu_sigma=True)              
XTrainKnn = cb.utils.knnimpute(XTrainScale, k=3)    

# Extract Test Data
XTest = DataTest[PeakList]                                     
XTestLog = np.log(XTest)                                          
XTestScale = cb.utils.scale(XTestLog, method='auto', mu=mu, sigma=sigma) 
XTestKnn = cb.utils.knnimpute(XTestScale, k=3)              

### 4. Hyperparameter Optimisation

In [4]:
# Parameter Dictionary
lr = [0.001,0.005,0.01,0.05,0.1,1]
neurons = [2, 3, 4, 5, 6]

param_dict = dict(learning_rate=lr,
                  n_neurons=neurons,
                  epochs=400,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')


# Initialise
cv = cb.cross_val.KFold(model=cb.model.NN_SigmoidSigmoid,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)                                

# Run and Plot
cv.run()  
cv.plot(metric='auc', color_beta=[5,5,5])
cv.plot(metric='r2q2', color_beta=[5,5,5])

Number of cores set to: 32
Running ...


1/2: 100%|████████████████████████████████████| 30/30 [00:00<00:00, 1486.52it/s]
Using Theano backend.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Using Theano backend.
INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '3602590')
Using Theano backend.
INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '3602607')
Using Theano backend.
INFO (theano.gof.compilelock): Waiting for existing lock by unknown process (I am process '3602591')
INFO (theano.gof.compilelock): To manually release the lock, delete /data/gpfs-1/home/users/phgi10_c/.theano/compiledir_Linux-4.18-el8_5.x86_64-x86_64-with-centos-8.7-Green_Obsidian-x86_64-3.7.16-64/lock_dir
INFO (theano.gof.compilelock): To manually release the lock, delete /data/gpfs-1/home/users/phgi10_c/.theano/compiledir_Linux-4.18-el8_5.x86

Time taken: 6.99 minutes with 32 cores
Done!


/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing i

Loading BokehJS ...

/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  x_init = scaler.fit_transform(x[:, np.newaxis]).flatten()
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/cimcb/utils/color_scale.py:9: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing i

Loading BokehJS ...

In [5]:
# Parameter Dictionary
lr = [0.001,0.01,0.02,0.04,0.06,0.08,0.1,0.2,0.5,1]

param_dict = dict(learning_rate=lr,
                  n_neurons=4,
                  epochs=400,
                  momentum=0.5,
                  decay=0,
                  loss='binary_crossentropy')

# Initialise
cv = cb.cross_val.KFold(model=cb.model.NN_SigmoidSigmoid,                      
                        X=XTrainKnn,                                 
                        Y=YTrain,                               
                        param_dict=param_dict,                   
                        folds=5,
                        n_mc=10)                             

# Run and Plot
cv.run()  
cv.plot(metric='auc')
cv.plot(metric='r2q2')

Number of cores set to: 32
Running ...


2/2:   0%|                                              | 0/100 [00:00<?, ?it/s]/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.118101). Check your callbacks.
  % delta_t_median)
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.136244). Check your callbacks.
  % delta_t_median)
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.160073). Check your callbacks.
  % delta_t_median)
/data/gpfs-1/users/phgi10_c/work/miniconda3/envs/aml_p1/lib/python3.7/site-packages/keras/callbacks.py:122: UserWarning: Method on_batch_end() is slow compared to the batch update (0.160061). Check your callbacks.
  % 

Time taken: 1.38 minutes with 32 cores
Done!


Loading BokehJS ...

Loading BokehJS ...

### 5. Build Model & Evaluate

In [6]:
# Build Model
model = cb.model.NN_SigmoidSigmoid(learning_rate=0.02, 
                                  n_neurons=4,
                                  epochs=400,
                                  momentum=0.5, 
                                  decay=0, 
                                  loss='binary_crossentropy')
YPredTrain = model.train(XTrainKnn, YTrain)
YPredTest = model.test(XTestKnn)

# Put YTrain and YPredTrain in a List
EvalTrain = [YTrain, YPredTrain]

# Put YTest and YPrestTest in a List
EvalTest = [YTest, YPredTest]

# Evaluate Model (include Test Dataset)
model.evaluate(testset=EvalTest) 

WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.


Loading BokehJS ...

### 6. Bootstrap Evaluation

In [7]:
# Extract X Data
XBoot = DataTable2[PeakList]
XBootLog = np.log(XBoot)
XBootScale = cb.utils.scale(XBootLog, method='auto')
XBootKnn = cb.utils.knnimpute(XBootScale, k=3)
YPredBoot = model.train(XBootKnn, Y)

# Build Boostrap Models
bootmodel = cb.bootstrap.Per(model, bootnum=100) 
bootmodel.run()

# Boostrap Evaluate Model (include Test Dataset)
bootmodel.evaluate(trainset=EvalTrain, testset=EvalTest) 

Number of cores set to: 32


100%|█████████████████████████████████████████| 100/100 [00:18<00:00,  5.52it/s]


Time taken: 0.71 minutes with 32 cores


Loading BokehJS ...

### 7. Export Results

In [8]:
home = 'results/'
file = 'ANNSigSig_MTBLS136.xlsx'

bootmodel.save_results(home + file)

Done! Saved results as results/ANNSigSig_MTBLS136.xlsx
